In [1]:
import cv2
import numpy as np
import openvino as ov

# WEBCAM

In [2]:
def Main():
    source = "data/cake.mp4"
    camera = cv2.VideoCapture(source)

    while True:
        ret, frame = camera.read()

        if not ret:
            break

        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    camera.release()
    cv2.destroyAllWindows()

# Call the Main function to start the program
if __name__ == "__main__":
    Main()


In [3]:
source = "data/cake.mp4"

if __name__ == '__name__':
    Main()

# Face Detection with Webcam

# Load Model

In [4]:
core = ov.Core()

model_face = core.read_model(model='Model/face-detection-adas-0001.xml')
compiled_model_face = core.compile_model(model = model_face, device_name= "CPU")

input_layer_face = compiled_model_face.input(0)
output_layer_face = compiled_model_face.output(0)

print("Input shape:", input_layer_face.shape)
print("Output shape:", output_layer_face.shape)

model_emo = core.read_model(model='Model/emotions-recognition-retail-0003.xml')
compiled_model_emo = core.compile_model(model = model_emo, device_name= "CPU")

input_layer_emo = compiled_model_emo.input(0)
output_layer_emo = compiled_model_emo.output(0)

print("Input shape:", input_layer_emo.shape)
print("Output shape:", output_layer_emo.shape)

model_ag = core.read_model(model='Model/age-gender-recognition-retail-0013.xml')
compiled_model_ag = core.compile_model(model = model_ag, device_name= "CPU")

input_layer_ag = compiled_model_ag.input(0)
output_layer_ag = compiled_model_ag.output(0)

print("Input shape:", input_layer_ag.shape)
print("Output shape:", output_layer_ag.shape)
compiled_model_ag = core.compile_model(model = model_ag, device_name= "CPU")

input_layer_ag = compiled_model_ag.input(0)
output_layer_ag = compiled_model_ag.output

print("Input shape:", input_layer_ag.shape)
print("Output shape:", output_layer_ag)

Input shape: [1,3,384,672]
Output shape: [1,1,200,7]
Input shape: [1,3,64,64]
Output shape: [1,5,1,1]
Input shape: [1,3,62,62]
Output shape: [1,2,1,1]
Input shape: [1,3,62,62]
Output shape: <bound method PyCapsule.output of <CompiledModel:
inputs[
<ConstOutput: names[data] shape[1,3,62,62] type: f32>
]
outputs[
<ConstOutput: names[prob] shape[1,2,1,1] type: f32>,
<ConstOutput: names[age_conv3, fc3_a] shape[1,1,1,1] type: f32>
]>>


### Pre-process New Input Data

In [5]:
def preprocess(frame, input_layer):
    N,input_channels,input_height, input_width = input_layer.shape
    
    resized_frame = cv2.resize(frame, (input_width, input_height))
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0 )
    
    return input_frame


### Postprocess the results

### Find the face boxes

In [6]:
def find_faceboxes(frame, results, confidence_threshold):
    results = results.squeeze()

    scores = results[:,2]
    boxes = results[:, -4:]

    face_boxes = boxes[scores >= confidence_threshold]
    scores = scores[scores >= confidence_threshold]

    frame_h, frame_w, frame_channels = frame.shape
    face_boxes = face_boxes*np.array([frame_w, frame_h, frame_w, frame_h])
    face_boxes = face_boxes.astype(np.int64)

    return face_boxes, scores

### Draw emotion/age/gender

In [7]:
def draw_age_gender_emotion(face_boxes, frame):

    EMOTION_NAMES = ['neutral', 'happy', 'sad', 'surprise', 'anger'] 

    show_frame = frame.copy()

    for i in range(len(face_boxes)):

        xmin, ymin, xmax, ymax = face_boxes[i]
        face = frame[ymin:ymax, xmin:xmax]
        
        # --- emotion ---
        input_frame = preprocess(face, input_layer_emo)
        results_emo = compiled_model_emo([input_frame])[output_layer_emo]

        results_emo.squeeze()
        index = np.argmax(results_emo)

        text = EMOTION_NAMES[index]
        # --- emotion ---

        # --- age and gender ---
        input_frame_ag = preprocess(face, input_layer_ag)
        results_ag = compiled_model_ag([input_frame_ag])
        age, gender = results_ag[1], results_ag[0]
        age = np.squeeze(age)
        age = int(age * 100)

        gender = np.squeeze(gender)

        if gender[0] >= 0.65:
            gender = "female"
            box_color = (200, 200, 0)
        elif gender[1] >= 0.55:
            gender = "male"
            box_color = (0, 200, 200)
        else:
            gender = "unknown"
            box_color = (200, 200, 200)
        
        if age < 30:
            box_color = (255, 0, 0)
        # --- age and gender ---

        fontScale = frame.shape[1] / 750
        
        text = gender + ' ' + str(age) + ' ' + EMOTION_NAMES[index]
        cv2.putText(show_frame, text, (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0, 200, 0), 8)
        cv2.rectangle(img=show_frame, pt1=(xmin, ymin), pt2=(xmax, ymax), color=box_color, thickness=2)

    return show_frame

# Run Inference

In [18]:
def Main():
    source = "data/cake.mp4"
    camera = cv2.VideoCapture(source)

    while True:
        ret, frame = camera.read()

        if not ret:
            break

        input_frame = preprocess(frame, input_layer_face)
        results = compiled_model_face([input_frame])[output_layer_face]

        face_boxes, scores = find_faceboxes(frame, results, confidence_threshold)
        show_frame = draw_age_gender_emotion(face_boxes, frame)

        cv2.imshow("Webcam", show_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    camera.release()
    cv2.destroyAllWindows()

confidence_threshold = .95
if __name__ == "__main__":
    Main()

In [15]:
confidence_threshold = .95
source = "data/cake.mp4"

if __name__ == '__name__':
    Main()